In [1]:
!pip install pandas


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import pandas as pd
import numpy as np
import math
import itertools

In [95]:
POSSIBLE_VALUES = set(range(1,10))
BLOQUES = [
    set(itertools.product([0,1,2], ['A', 'B', 'C'])),
    set(itertools.product([0,1,2], ['D', 'E', 'F'])),
    set(itertools.product([0,1,2], ['G', 'H', 'I'])),
    
    set(itertools.product([3,4,5], ['A', 'B', 'C'])),
    set(itertools.product([3,4,5], ['D', 'E', 'F'])),
    set(itertools.product([3,4,5], ['G', 'H', 'I'])),
    
    set(itertools.product([6,7,8], ['A', 'B', 'C'])),
    set(itertools.product([6,7,8], ['D', 'E', 'F'])),
    set(itertools.product([6,7,8], ['G', 'H', 'I'])),
]
BLOQUES

[{(0, 'A'),
  (0, 'B'),
  (0, 'C'),
  (1, 'A'),
  (1, 'B'),
  (1, 'C'),
  (2, 'A'),
  (2, 'B'),
  (2, 'C')},
 {(0, 'D'),
  (0, 'E'),
  (0, 'F'),
  (1, 'D'),
  (1, 'E'),
  (1, 'F'),
  (2, 'D'),
  (2, 'E'),
  (2, 'F')},
 {(0, 'G'),
  (0, 'H'),
  (0, 'I'),
  (1, 'G'),
  (1, 'H'),
  (1, 'I'),
  (2, 'G'),
  (2, 'H'),
  (2, 'I')},
 {(3, 'A'),
  (3, 'B'),
  (3, 'C'),
  (4, 'A'),
  (4, 'B'),
  (4, 'C'),
  (5, 'A'),
  (5, 'B'),
  (5, 'C')},
 {(3, 'D'),
  (3, 'E'),
  (3, 'F'),
  (4, 'D'),
  (4, 'E'),
  (4, 'F'),
  (5, 'D'),
  (5, 'E'),
  (5, 'F')},
 {(3, 'G'),
  (3, 'H'),
  (3, 'I'),
  (4, 'G'),
  (4, 'H'),
  (4, 'I'),
  (5, 'G'),
  (5, 'H'),
  (5, 'I')},
 {(6, 'A'),
  (6, 'B'),
  (6, 'C'),
  (7, 'A'),
  (7, 'B'),
  (7, 'C'),
  (8, 'A'),
  (8, 'B'),
  (8, 'C')},
 {(6, 'D'),
  (6, 'E'),
  (6, 'F'),
  (7, 'D'),
  (7, 'E'),
  (7, 'F'),
  (8, 'D'),
  (8, 'E'),
  (8, 'F')},
 {(6, 'G'),
  (6, 'H'),
  (6, 'I'),
  (7, 'G'),
  (7, 'H'),
  (7, 'I'),
  (8, 'G'),
  (8, 'H'),
  (8, 'I')}]

In [96]:
POSSIBLE_VALUES

{1, 2, 3, 4, 5, 6, 7, 8, 9}

In [112]:
sudoku_df = pd.read_csv('sudoku_input.csv',  header=None)
sudoku_df.columns = ['A', 'B', 'C','D','E','F','G','H','I']
sudoku_df.head(10)

,A,B,C,D,E,F,G,H,I
0,5.0,3.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN
1,6.0,NaN,NaN,1.0,9.0,5.0,NaN,NaN,NaN
2,NaN,9.0,8.0,NaN,NaN,NaN,NaN,6.0,NaN
3,8.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,3.0
4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
6,NaN,6.0,NaN,NaN,NaN,NaN,2.0,8.0,4.0
7,NaN,NaN,NaN,4.0,1.0,9.0,NaN,3.0,5.0
8,NaN,NaN,NaN,NaN,8.0,NaN,NaN,7.0,9.0


In [113]:
sudoku_df = sudoku_df.fillna('')
sudoku_df.head()

,A,B,C,D,E,F,G,H,I
0,5.0,3.0,,,7.0,,,,
1,6.0,,,1.0,9.0,5.0,,,
2,,9.0,8.0,,,,,6.0,
3,8.0,,,,6.0,,,,3.0
4,4.0,,,,,,,,1.0


In [114]:
def get_indexes(row_index, column_index):
    index_blocks = [b for b in BLOQUES if (row_index, column_index) in b][0]
    column_indexes = {(row_index, c_index) for c_index in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']}
    row_indexes = {(r_index, column_index) for r_index in range(9)}
    
    return index_blocks.union(column_indexes).union(row_indexes)

In [115]:
def get_values_in_indexes(indexes, df):
    values_set = set()
    for r_i, c_i in indexes:
        value_cell = df.iloc[r_i][c_i]
        if not (isinstance(value_cell, str) or isinstance(value_cell, set)):
            values_set = values_set.union({value_cell})
    return values_set

In [116]:
def get_df_sudoku(sudoku_df_):
    sudoku_df_ = sudoku_df_.copy()
    for row_index, row in sudoku_df_.iterrows():
        for c_index in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
            cell_value_is_empty = sudoku_df_.iloc[row_index][c_index] ==  ''
            if cell_value_is_empty:
                cell_possible_values = POSSIBLE_VALUES
                used_values = get_values_in_indexes(get_indexes(row_index, c_index), sudoku_df_)
                cell_possible_values = cell_possible_values - used_values
                if len(cell_possible_values) == 0:
                    return None
                if len(cell_possible_values) == 1:
                    sudoku_df_.iloc[row_index][c_index] = cell_possible_values.pop()
                else:
                    sudoku_df_.iloc[row_index][c_index] = cell_possible_values
    return sudoku_df_   

In [123]:
sudoku_df_v1 = get_df_sudoku(sudoku_df)
sudoku_df_v1

,A,B,C,D,E,F,G,H,I
0,5.0,3.0,"{1, 2, 4}","{8, 2, 6}",7.0,"{8, 2, 4, 6}","{8, 1, 4, 9}","{1, 2, 4, 9}","{8, 2}"
1,6.0,"{2, 4, 7}","{2, 4, 7}",1.0,9.0,5.0,"{8, 3, 4, 7}","{2, 4}","{8, 2, 7}"
2,"{1, 2}",9.0,8.0,"{2, 3}","{2, 3, 4}","{2, 3, 4}","{1, 3, 4, 5, 7}",6.0,"{2, 7}"
3,8.0,"{1, 2, 5}","{1, 2, 5, 9}","{9, 2, 5, 7}",6.0,"{1, 2, 4, 7}","{9, 4, 5, 7}","{9, 2, 4, 5}",3.0
4,4.0,"{2, 5}","{2, 3, 5, 6, 9}","{2, 3, 5, 7, 8, 9}","{2, 3, 5}","{8, 2, 3, 7}","{8, 9, 5, 7}","{9, 2, 5}",1.0
5,7.0,"{1, 2, 5}","{1, 2, 3, 5, 9}","{2, 3, 5, 8, 9}","{2, 3, 4, 5}","{1, 2, 3, 4, 8}","{8, 9, 4, 5}","{9, 2, 4, 5}",6.0
6,"{1, 3, 9}",6.0,"{1, 3, 5, 7, 9}","{3, 5, 7}","{3, 5}","{3, 7}",2.0,8.0,4.0
7,2,"{8, 7}",7,4.0,1.0,9.0,6,3.0,5.0
8,"{1, 3}","{1, 4, 5}","{1, 3, 4, 5}","{2, 3, 5, 6}",8.0,"{2, 3, 6}",1,7.0,9.0


3